## 欢迎进入 ModelWhale Notebook  

这里你可以编写代码，文档  

### 关于文件目录  


**project**：project 目录是本项目的工作空间，可以把将项目运行有关的所有文件放在这里，目录中文件的增、删、改操作都会被保留  


**input**：input 目录是数据集的挂载位置，所有挂载进项目的数据集都在这里，未挂载数据集时 input 目录被隐藏  


**temp**：temp 目录是临时磁盘空间，训练或分析过程中产生的不必要文件可以存放在这里，目录中的文件不会保存  


## 爬取微博上“科技改变生活”相关文章

In [7]:
#from lxml import etree
import requests
import random
import time
import csv
import pandas as pd 
import numpy as np
import re

In [8]:
# 设置请求头参数：User-Agent, cookie, referer

headers = {
    # 随机生成User-Agent
    'User-Agent' : "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36", #"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36",
    # 不同用户不同时间访问，cookie都不一样，根据自己网页的来，获取方法见Python之反爬虫手段
    'cookie' : '_T_WM=54639083573; WEIBOCN_FROM=1110006030; MLOGIN=0; XSRF-TOKEN=d0b7c1; M_WEIBOCN_PARAMS=luicode%3D10000011%26lfid%3D100103type%253D1%2526q%253D%25E6%259C%25BA%25E7%25A5%25A8%26fid%3D100103type%253D1%2526q%253D%25E6%259C%25BA%25E7%25A5%25A8%26uicode%3D10000011',
    # 设置从何处跳转过来(本次可以不用设置)
    #'referer': '',
}

In [9]:
columns = ['网名', '国家', '省会', '城市', '手机终端', '文本']
df = pd.DataFrame(columns=columns)

In [10]:
# 假设这里有 headers 变量，你需要根据实际情况设置
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}

# 初始化一个空的 DataFrame
df = pd.DataFrame(columns=['网名', '国家', '省会', '城市', '手机终端', '文本'])

for num in range(100):  # 由于时间原因，只爬取100页，约1000条左右数据
    # 首页网址URL
    #机票
    #url = 'https://m.weibo.cn/api/container/getIndex?containerid=100103type%3D61%26q%3D%E6%9C%BA%E7%A5%A8%26t%3D&page_type=searchall&page=' + str(num + 1)
    # print(url)
    url = 'https://m.weibo.cn/api/container/getIndex?containerid=100103type%3D61%26q%3D%E7%A7%91%E6%8A%80%E6%94%B9%E5%8F%98%E7%94%9F%E6%B4%BB%26t%3D&page_type=searchall' + str(num + 1)
    # print(url)

    # time.sleep(1)
    # 请求发送
    response = requests.get(url=url, headers=headers).json()
    card_list = response.get('data').get('cards')
    print("******************开始爬取第", str(num + 1), "页******************")
    temp_dfs = []  # 用于存储临时的 DataFrame
    for i in range(len(card_list)):
        values = []
        if 'card_group' in card_list[i] and 'mblog' in card_list[i]:
            dict1 = card_list[i].get('card_group')[0].get('mblog')
            values.append(dict1.get('user').get('screen_name'))  # 网名
            values.append(dict1.get('status_country'))  # 国家
            values.append(dict1.get('status_province'))  # 省会
            values.append(dict1.get('status_city'))  # 城市
            values.append(dict1.get('source'))  # 手机终端
            values.append(re.sub("[A-Za-z0-9\!\%\[\]\,\。\<\=\"\:\/\.\/\?\&\-\>]", "", dict1.get('text')))  # 文本
            temp_df = pd.DataFrame([values], columns=df.columns)
            temp_dfs.append(temp_df)
            print(values)

        elif 'mblog' in card_list[i]:
            dict1 = card_list[i].get('mblog')
            values.append(dict1.get('user').get('screen_name'))  # 网名
            values.append(dict1.get('status_country'))  # 国家
            values.append(dict1.get('status_province'))  # 省会
            values.append(dict1.get('status_city'))  # 城市
            values.append(dict1.get('source'))  # 手机终端
            values.append(re.sub("[A-Za-z0-9\!\%\[\]\,\。\<\=\"\:\/\.\/\?\&\-\>\_\; \_\ ; \']", "", dict1.get('text')))  # 文本
            temp_df = pd.DataFrame([values], columns=df.columns)
            temp_dfs.append(temp_df)
            print(values)

    if temp_dfs:
        df = pd.concat([df] + temp_dfs, ignore_index=True)

    print("******************第", str(num + 1), "页爬取成功******************")

print(df)

******************开始爬取第 1 页******************
['用户7573613035008', '中国', '重庆', '重庆', '荣耀X50', '问界带你春游：一场科技与自然的邂逅寒冬褪去，万物复苏，正是春游好时节想要一场说走就走的旅行，又不想被传统燃油车的局限所束缚？问界，这款集科技与豪华于一身的智能电动车，将带你开启一场别样的春游体验打开全景天窗，让温暖的阳光洒满车内，呼吸着清晨新鲜的空气，全文']
['在屋檐下学唱歌的永丰', '中国', '山东', '烟台', 'HarmonyOS设备', '最近想减肥不知道怎么吃，上橙篇问了一下，居然还真给了我一周的减肥食谱科技改变生活啊真是']
['不凋谢月光X', '中国', '河南', '郑州', 'Android', '高科技改变生活']
['乐悠悠结城美柑1985j1976', '中国', '海南', '海口', 'HUAWEI P40 Pro 5G', '#懂事的男友已开始安排美甲#原本以为换美甲得去美甲店挑半天，现在男友用两分钟就搞定了，真是科技改变生活啊！']
['总裁视野', '中国', '重庆', '重庆', '', '#问界开启预订万元起#科技发展让汽车不再只是交通工具，更像是生活的好帮手感受它带来的科技魅力，体验科技改变生活的力量#余承东称问界是全能车王#']
['姓帅的乐山人', '中国', '四川', '成都', 'iPhone客户端', '大疆无人机新产品到店开箱仪式！机场开启无人机智能运维新时代！！++#数码科技#科技改变生活#大疆无人机成都·成都数码广场姓帅的乐山人的微博视频']
['长宁发布', '中国', '四川', '宜宾', '微博网页版', '【李琴：乡村沃野奏响青春“奋进乐章”】#长宁#在宜宾市长宁县花滩镇大冲村的广袤田野间，李琴的名字家喻户晓，乡亲们谈及她，无不竖起大拇指称赞这位曾在城市拥有安稳工作的青年，毅然决然踏上返乡创业之路，投身肉兔养殖产业她不仅实现了自我人生价值的升华，更如同一盏明灯，引领着乡亲们奔全文']
['_Sylviawu', '中国', '上海', '上海', 'iPhone客户端', '杭州公司出差话说杭州东网约车好方便，科技改变生活']
['上海文艺出版社', '中国', '上海', '上海', '微博网页版'

In [11]:
df.head() # 展示前5行

,网名,国家,省会,城市,手机终端,文本
0,用户7573613035008,中国,重庆,重庆,荣耀X50,问界带你春游：一场科技与自然的邂逅寒冬褪去，万物复苏，正是春游好时节想要一场说走就走的旅行，...
1,在屋檐下学唱歌的永丰,中国,山东,烟台,HarmonyOS设备,最近想减肥不知道怎么吃，上橙篇问了一下，居然还真给了我一周的减肥食谱科技改变生活啊真是
2,不凋谢月光X,中国,河南,郑州,Android,高科技改变生活
3,乐悠悠结城美柑1985j1976,中国,海南,海口,HUAWEI P40 Pro 5G,#懂事的男友已开始安排美甲#原本以为换美甲得去美甲店挑半天，现在男友用两分钟就搞定了，真是科...
4,总裁视野,中国,重庆,重庆,,#问界开启预订万元起#科技发展让汽车不再只是交通工具，更像是生活的好帮手感受它带来的科技魅力...


In [15]:
#df.to_csv('微博科技改变生活相关文章.csv')
file_name = '微博科技改变生活相关文章.csv'
df.to_csv(file_name,encoding="utf_8_sig")

## 情感分析

In [ ]:
#情感分析
#需要分析的数据集
data=pd.read_excel("./美团.xls")
print(data.head())

In [ ]:
# 查看数据集大小
data.shape

In [ ]:
#  查看索引、数据类型和内存信息
data.info()

In [ ]:
# 对重复数据做删除处理
data = data.drop_duplicates(inplace=False)
data

In [ ]:
# 此处虽然删除了重复值，但索引未变，因此应用以下方法进行重置索引
data.reset_index(inplace=True,drop=True)

In [ ]:
data

In [ ]:
# 查看缺失值
data.isnull().any()
# 查看数据结构
data.describe()
# 查看评分列的众数
mode_01 = data.评分.mode()
mode_01
data = data.fillna(0)
data.isnull().sum()

In [ ]:
import jieba
# jieba.load_userdict('addwords.txt')
title_cut = []
for i in data.标题:
    j = jieba.lcut(i)
    title_cut.append(j)

In [ ]:
title_cut

In [ ]:
file_path = open(r'./停用词表.txt',encoding='utf-8')
stop_words = file_path.read()
stop_words

In [ ]:
# 新建一个空列表，用于存储删除停用词后的数据
new_data = []
for word in cut_words:
    if word not in stop_words:
        new_data.append(word)
print(new_data)

In [ ]:
from nltk import FreqDist
freq_list = FreqDist(new_data)
# 返回词语列表
most_common_words = freq_list.most_common()
print(most_common_words)

In [ ]:
from pyecharts import options as opts
from pyecharts.charts import Page, WordCloud
from pyecharts.globals import SymbolType
def wordcloud_base() -> WordCloud:
    c = (
        WordCloud()
        .add("", most_common_words, word_size_range=[20, 100])
        .set_global_opts(title_opts=opts.TitleOpts(title="美团标题标签可视化"))
    )
    return c

wd = wordcloud_base()
wd.render_notebook()

In [ ]:
data["分类"].unique()

In [ ]:
brand_data2 = data.groupby('分类')['评分'].agg(['mean', 'median'])

In [ ]:
brand_data2